In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import pickle
from scipy import sparse,spatial,stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip

--2021-09-20 09:01:30--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  10.1MB/s    in 0.5s    

2021-09-20 09:01:31 (10.1 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [ ]:
!unzip ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [ ]:
trainData = pd.read_csv("ml-100k/u1.base",sep='\t',names=['user_id','item_id','rating','timestamp'])

In [ ]:
utilityMatrix=sparse.load_npz("utilityMatrix.npz")

In [ ]:
movieSets = dict.fromkeys(range(1,1683))
for key in movieSets.keys():
  movieSets[key]=set()

for index, row in trainData.iterrows():
    if row['user_id'] not in movieSets[row['item_id']]:
      movieSets[row['item_id']].add(row['user_id'])

In [ ]:
#!!
userSets = dict.fromkeys(range(944))
for key in userSets.keys():
  userSets[key]=set()

for index, row in trainData.iterrows():
    
    if row['item_id'] not in userSets[row['user_id']]:
      # print(row['user_id'],row['item_id'],row['rating'])
      userSets[row['user_id']].add(row['item_id'])

In [ ]:
def is_relevant(rating):
  if rating>3.5:
    return 1
  else:
    return 0

def average_precision_k(dataset,k=10,threshold=3):
  # dataset['relevant_actual']=dataset['rating'].map(is_relevant)
  # dataset['relevant_predicted']=dataset['predicted_rating'].map(is_relevant)
  precisions=[]
  for user in range(943):
    dataframeUser = dataset[dataset['user_id']==user+1]
    dataframeUserActual = dataframeUser.sort_values(by='rating',ascending=False)
    dataframeUserPredicted = dataframeUser.sort_values(by='predicted_rating',ascending=False)
    actualReleventSet=set()
    predictedReleventSet=set()
    actualCounter=0
    predictedCounter=0
    for index,row in dataframeUserActual.iterrows():
      if(actualCounter==k):
        break
      if(row['rating']>=threshold):
        actualCounter+=1
        actualReleventSet.add(row['item_id'])
    for index,row in dataframeUserPredicted.iterrows():
      if(predictedCounter==k):
        break
      if(row['predicted_rating']>=threshold):
        predictedCounter+=1
        predictedReleventSet.add(row['item_id'])   
    if(len(actualReleventSet)==0):
      userPrecisionK==1
    else:
      userPrecisionK = len(actualReleventSet & predictedReleventSet)/len(actualReleventSet)   
    precisions.append(userPrecisionK)
  return sum(precisions)/len(precisions)

**1. User-User**

In [ ]:
PearsonCorrelation = pd.DataFrame(utilityMatrix.transpose().toarray()).corr(method='pearson').values
PearsonCorrelation = sparse.csr_matrix(PearsonCorrelation)

In [ ]:
########## N most similar users who rated the same movie
def MostSimilarUser(M,movie,user,n):
  movieUserList=movieSets[movie]
  similarUserList=[i+1 for i in list(np.argsort(M[user].toarray())[0][::-1])]
  nSimilarUsersRatedMovie=[]
  counter=0
  for similarUser in similarUserList:
    if similarUser in movieUserList:
      nSimilarUsersRatedMovie.append(similarUser)
      counter+=1
    if counter==n:
      break
  return nSimilarUsersRatedMovie

In [ ]:
########## Rating Function
def calculateRatingUserUser1(M,movie,user,n):
  listSimilarUsers=MostSimilarUser(M,movie,user,n)
  similarUsersRatings=0
  for similarUser in listSimilarUsers:
    similarUsersRatings+=utilityMatrix[similarUser-1,movie-1]
  averageRating=similarUsersRatings/float(n)
  return averageRating
  
def calculateRatingUserUser2(M,movie,user,n):
  listSimilarUsers=MostSimilarUser(M,movie,user,n)
  weightedSimilaritySum=0
  similaritySum=0
  for similarUser in listSimilarUsers:
    weightedSimilaritySum+=M[user-1,similarUser-1]*utilityMatrix[similarUser-1,movie-1]
    similaritySum+=M[user-1,similarUser-1]
  if (similaritySum==0):
    rating=0
  else:
    rating=weightedSimilaritySum/similaritySum
  return rating


def calculateRatingUserUser3(M,movie,user,n):
  listSimilarUsers=MostSimilarUser(M,movie,user,n)
  weightedSimilaritySum=0
  similaritySum=0
  movieArray=utilityMatrix[:,movie]
  movieMean=np.mean(movieArray[movieArray.nonzero()])

  userArray=utilityMatrix[user,:]
  userMean=np.mean(userArray[userArray.nonzero()])
  userDeviation=userMean-movieMean
  movieDeviation=utilityMatrix[user,movie]-movieMean
  bias = movieMean + userDeviation + movieDeviation
  for similarUser in listSimilarUsers:
    if(similarUser>940):
      continue
    #######
    similaruserArray=utilityMatrix[similarUser,:]
    similaruserMean=np.mean(similaruserArray[similaruserArray.nonzero()])
    similaruserDeviation=similaruserMean-movieMean
    similarUserbias = movieMean + similaruserDeviation + movieDeviation
    #######
    weightedSimilaritySum+=M[user-1,similarUser-1]*(utilityMatrix[similarUser-1,movie-1]-similarUserbias)
    #######
    similaritySum+=M[user-1,similarUser-1]
  if (similaritySum==0):
    rating=0
  else:
    rating=weightedSimilaritySum/similaritySum
  return bias+rating


In [ ]:
testData = pd.read_csv("ml-100k/u1.test",sep='\t',names=['user_id','item_id','rating','timestamp'])

1.1 Simple Average Rating


1.1.1 Evaluation(n=2)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser1(PearsonCorrelation,row['item_id'],row['user_id'],2)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.2522879061940988

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.24423740510697028

1.1.2 Evaluation(n=5)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser1(PearsonCorrelation,row['item_id'],row['user_id'],5)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.1419439565933172

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.7577016950293726

1.1.3 Evaluation(n=10)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser1(PearsonCorrelation,row['item_id'],row['user_id'],10)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.108171918070477

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.2543949569930482

In [ ]:
testData = pd.read_csv("ml-100k/u1.test",sep='\t',names=['user_id','item_id','rating','timestamp'])

1.2 Weighted Average Rating


1.2.1 Evaluation(n=2)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser2(PearsonCorrelation,row['item_id'],row['user_id'],2)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

2.065238576055945

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.2323141274891008

1.2.2 Evaluation(n=5)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser2(PearsonCorrelation,row['item_id'],row['user_id'],5)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.720197037003922

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.7544955309801544

1.2.3 Evaluation(n=10)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser2(PearsonCorrelation,row['item_id'],row['user_id'],10)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.108171918070477

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.2543949569930482

1.3 Weighted Average with bias Rating


1.3.1 Evaluation(n=2)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser3(PearsonCorrelation,row['item_id'],row['user_id'],2)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/base.py:585: RuntimeWarning: divide by zero encountered in true_divide
  return self.astype(np.float_)._mul_scalar(1./other)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/base.py:585: RuntimeWarning: divide by zero encountered in true_divide
  return self.astype(np.float_)._mul_scalar(1./other)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/base.py:585: RuntimeWarning: divide by zero encountered in true_divide
  return self.astype(np.float_)._mul_scalar(1./other)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/base.py:585: RuntimeWarning: divide by zero encountered in true_divide
  return self.astype(np.float_)._mul_scalar(1./other)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/base.py:585: RuntimeWarning: divide by zero encountered in true_divide
  return self.astype(np.float_)._mul_scalar(1./other)
/usr/local/lib/python3.7/dist-packages/scipy/sparse/base.py:585: RuntimeWarning: divide by zero encountered in true

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

3.064774113405453

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.21733912370179592

1.3.2 Evaluation(n=5)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser2(PearsonCorrelation,row['item_id'],row['user_id'],5)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.720197037003922

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.7544955309801544

1.3.3 Evaluation(n=10)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingUserUser2(PearsonCorrelation,row['item_id'],row['user_id'],10)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.108171918070477

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.2543949569930482

In [ ]:
header=pd.MultiIndex.from_product([['User-User'],
                                     ['Simple Average','Weighted Average','Weighted Average with Bias'],['RMSE','Precision@10'],['N=2','N=5','N=10']],
                                    names=['Algorithm','Rating Function','Metric','Neighbours'])
resultsTable=pd.DataFrame(np.reshape(np.array([1.25,1.14,1.10,0.24,0.75,0.25,2.06,1.72,1.10,0.23,0.75,0.25,3.06,1.72,1.108,0.21,0.75,0.25]),(1,18)),columns=header)

In [ ]:
display(resultsTable)

Algorithm            User-User             ...                                       
Rating Function Simple Average             ... Weighted Average with Bias            
Metric                    RMSE             ...               Precision@10            
Neighbours                 N=2   N=5 N=10  ...                        N=2   N=5  N=10
0                         1.25  1.14  1.1  ...                       0.21  0.75  0.25

[1 rows x 18 columns]

**2. Item-Item**

In [ ]:
PearsonCorrelation = pd.DataFrame(utilityMatrix.toarray()).corr(method='pearson').values
PearsonCorrelation = sparse.csr_matrix(PearsonCorrelation)

In [ ]:
########## N most similar movies
def MostSimilarMovies(M,movie,user,n):
  usersMovieList=userSets[user]
  similarMovieList=[i+1 for i in list(np.argsort(M[movie].toarray())[0][::-1])]
  nSimilarMoviesRatedByUser=[]
  counter=0
  for similarMovie in similarMovieList:
    if similarMovie in usersMovieList:
      nSimilarMoviesRatedByUser.append(similarMovie)
      counter+=1
    if counter==n:
      break
  return nSimilarMoviesRatedByUser

In [ ]:
########## Rating Function
def calculateRatingItemItem1(M,movie,user,n):
  listSimilarMovies=MostSimilarMovies(M,movie,user,n)
  similarMoviesRatings=0
  for similarMovie in listSimilarMovies:
    similarMoviesRatings+=utilityMatrix[user-1,similarMovie-1]
  averageRating=similarMoviesRatings/float(n)
  return averageRating
  
def calculateRatingItemItem2(M,movie,user,n):
  listSimilarMovies=MostSimilarMovies(M,movie,user,n)
  weightedSimilaritySum=0
  similaritySum=0
  for similarMovie in listSimilarMovies:
    weightedSimilaritySum+=M[movie-1,similarMovie-1]*utilityMatrix[user-1,similarMovie-1]
    similaritySum+=M[movie-1,similarMovie-1]
  if (similaritySum==0):
    rating=0
  else:
    rating=weightedSimilaritySum/similaritySum
  return rating


def calculateRatingItemItem3(M,movie,user,n):
  listSimilarMovies=MostSimilarMovies(M,movie,user,n)
  weightedSimilaritySum=0
  similaritySum=0
  movieArray=utilityMatrix[:,movie]
  movieMean=np.mean(movieArray[movieArray.nonzero()])

  userArray=utilityMatrix[user,:]
  userMean=np.mean(userArray[userArray.nonzero()])
  userDeviation=userMean-movieMean
  movieDeviation=utilityMatrix[user,movie]-movieMean
  bias = movieMean + userDeviation + movieDeviation
  for similarMovie in listSimilarMovies:
    if(similarMovie>1680):
      continue
    #######
    similarmovieArray=utilityMatrix[:,similarMovie]
    similarmovieMean=np.mean(similarmovieArray[similarmovieArray.nonzero()])
    similarmovieDeviation=similarmovieMean-movieMean
    similarMoviebias = movieMean + similarmovieDeviation + movieDeviation
    #######
    weightedSimilaritySum+=M[movie-1,similarMovie-1]*(utilityMatrix[user-1,similarMovie-1]-similarMoviebias)
    #######
    similaritySum+=M[movie-1,similarMovie-1]
  if (similaritySum==0):
    rating=0
  else:
    rating=weightedSimilaritySum/similaritySum
  return bias+rating


In [ ]:
testData = pd.read_csv("ml-100k/u1.test",sep='\t',names=['user_id','item_id','rating','timestamp'])

2.1 Simple Average Rating


2.1.1 Evaluation(n=2)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingItemItem1(PearsonCorrelation,row['item_id'],row['user_id'],2)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.2525474042925482

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.7413998720732549

2.1.2 Evaluation(n=5)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingItemItem1(PearsonCorrelation,row['item_id'],row['user_id'],5)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.1356751296035323

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.23388333417495644

2.1.3 Evaluation(n=10)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingItemItem1(PearsonCorrelation,row['item_id'],row['user_id'],10)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

1.091602262731257

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.7465173626891548

2.2 Weighted Average Rating


1.2.1 Evaluation(n=2)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingItemItem2(PearsonCorrelation,row['item_id'],row['user_id'],2)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

12.546825995028968

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.22284965240956753

1.2.2 Evaluation(n=5)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingItemItem2(PearsonCorrelation,row['item_id'],row['user_id'],5)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

4.454566102200838

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.23114889999831661

1.2.3 Evaluation(n=10)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingItemItem2(PearsonCorrelation,row['item_id'],row['user_id'],10)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

20.68206732667912

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.7502625864767963

1.3 Weighted Average with bias Rating


1.3.1 Evaluation(n=2)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingItemItem3(PearsonCorrelation,row['item_id'],row['user_id'],2)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

14.921508314205845

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.7238949485768147

1.3.2 Evaluation(n=5)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingItemItem3(PearsonCorrelation,row['item_id'],row['user_id'],5)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

4.356024619462531

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.2233171741655301

1.3.3 Evaluation(n=10)

In [ ]:
testData['predicted_rating']=[0 for _ in range(testData.shape[0])]
for index,row in testData.iterrows():
    rating_=calculateRatingItemItem3(PearsonCorrelation,row['item_id'],row['user_id'],10)
    testData.loc[index,'predicted_rating']=rating_
testData['predicted_rating']=testData['predicted_rating'].fillna(0)

==> Root Mean Squared Error

In [ ]:
#### Root mean squared error n=2
mean_squared_error(testData['rating'],testData['predicted_rating'],squared=False)

22.13619923594598

==> Precision@10

In [ ]:
average_precision_k(testData,k=10)

0.22537578481374856

In [ ]:
header=pd.MultiIndex.from_product([['Item-Item'],
                                     ['Simple Average','Weighted Average','Weighted Average with Bias'],['RMSE','Precision@10'],['N=2','N=5','N=10']],
                                    names=['Algorithm','Rating Function','Metric','Neighbours'])
resultsTable=pd.DataFrame(np.reshape(np.array([1.25,1.13,1.09,0.74,0.23,0.74,12.54,4.45,20.68,0.22,0.23,0.75,14.92,4.35,22.13,0.72,0.22,0.22]),(1,18)),columns=header)

In [ ]:
display(resultsTable)

Algorithm            Item-Item              ...                                       
Rating Function Simple Average              ... Weighted Average with Bias            
Metric                    RMSE              ...               Precision@10            
Neighbours                 N=2   N=5  N=10  ...                        N=2   N=5  N=10
0                         1.25  1.13  1.09  ...                       0.72  0.22  0.22

[1 rows x 18 columns]